In [ ]:
#
# Data I/O routines
# 
import numpy as np
import pandas as pds
import sys
import re
import os
from itertools import product

# Database & I/O routines for ML fits 
#   dpar: dictionary of parameters
#   key:  Output directory & prefix
class Data_IO:
    def __init__(self, dpar, key):
        if key is not None:
            self.ID = str(key)+'.'
        else:
            self.ID = ''
        # data file format: 'pdf' for Chroma ascii files; 'raw' for 1-d list of X & Y
        self.flist = [ 'pdf', 'raw']
        self.form = dpar['format']
        assert(self.form in self.flist)
        try:
            self.dbin = dpar['binsize']
        except:
            self.dbin = 0
        try:
            self.emax = dpar['emax']
        except:
            self.emax = None
        try:
            self.nsrc = dpar['nsrc']
        except:
            self.nsrc = 1
        try:
            self.NA = str(dpar['is2pt'])
        except:
            self.NA = str(None)
        try:
            self.reverse = dpar['reverse']
        except:
            self.reverse = False
        self.make_data(dpar)
        try:
            self.odir = dpar['odir']
            self.osave = True
            self.dsave = True
        except:
            self.osave = False
            self.dsave = False
            self.odir = None
        self.setup_output()
        return
    
    # Read in data files and generate database
    def make_data(self, dpar):
        self.ndata = {}
        try:
            self.ddir = dpar['ddir']
            for d1, d2, f in os.walk(self.ddir): 
                self.xfile = [self.ddir+'/'+ff for ff in f]
        except:
            self.ddir = None
            self.xfile = dpar['dfile.x']
            self.yfile = dpar['dfile.y']
            if self.form == 'pdf':
                self.xfile.extend(self.yfile)
        if self.form == 'pdf':
            self.NT = {}
            self.p = []
            self.z = []
            self.nx = 0
            self.ny = 0
            self.pztag = []
            self.pdict = {}
            self.zdict = {}
            self.tdict = {}
            self.tdict['X'] = None
            self.tdict['Y'] = None
            self.pdict['X'] = None
            self.pdict['Y'] = None
            self.zdict['X'] = None
            self.zdict['Y'] = None
            dataframe = {}
            for dfile in self.xfile:
                mode = re.search('((?<=pz)|(?<=p))((\d+)|(-\d+))', dfile)
                try:
                    self.p.append(str(mode.group(0)))
                except:
                    print("Unknown PDF data filename convention for momentum p/pz\n")
                    sys.exit(1)
                mode = re.search('(?<=z)(?<!pz)((\d+)|(-\d+))', dfile)
                try:
                    self.z.append(str(mode.group(0)))
                except:
                    # 2pt data: z = None (2pt)
                    self.z.append(self.NA)
                r = 1
                i = 2
                if self.reverse:
                    r = 2
                    i = 1
                # loop over t & configs.
                self.pztag.append(self.p[-1]+'+'+self.z[-1])
                tagX = self.pztag[-1]
                with open(dfile, 'r') as df:
                    n = 0
                    if self.dbin > 1:
                        bkid = 0
                        datblk = {}
                    for line in df:
                        tl = line.split()
                        if n == 0:
                            self.ndata[tagX] = int(tl[0])
                            self.NT[tagX] = int(tl[1])
                            self.nx += 2*self.NT[tagX]
                            # generate columns : p+z+t+{R,I}
                            for tt in range(self.NT[tagX]):
                                dataframe[tagX+'+t+'+str(tt)+'+R'] = []
                                dataframe[tagX+'+t+'+str(tt)+'+I'] = []
                                if self.dbin > 1:
                                    datblk[str(tt)] = [0., 0.]
                            n+=1
                            continue
                        if self.dbin == 1:
                            dataframe[tagX+'+t+'+tl[0]+'+R'].append(float(tl[r]))
                            dataframe[tagX+'+t+'+tl[0]+'+I'].append(float(tl[i]))
                            continue
                        datblk[tl[0]][0] += float(tl[r])
                        datblk[tl[0]][1] += float(tl[i])
                        if bkid == self.dbin-1:
                            dataframe[tagX+'+t+'+tl[0]+'+R'].append(datblk[tl[0]][0]/self.dbin)
                            dataframe[tagX+'+t+'+tl[0]+'+I'].append(datblk[tl[0]][1]/self.dbin)
                            datblk[tl[0]] = [0., 0.]
                        if int(tl[0]) == self.NT[tagX]-1:
                            bkid = (bkid+1) % self.dbin
            self.data = pds.DataFrame(data=dataframe)
            del dataframe
        elif self.form == 'raw':
            pdfile = open(self.xfile, 'r')
            data = {}
            data['X'] = np.fromfile(pdfile, dtype=float, sep=" ")
            pdfile.close()
            pdfile = open(self.yfile, 'r')
            data['Y'] = np.fromfile(pdfile, dtype=float, sep=" ")
            pdfile.close()
            self.ndata['Y'] = data['Y'].shape[0]
            self.ndata['X'] = self.ndata['Y']
            self.nx = int(data['X'].shape[0]/self.ndata['X'])
            data['X'].shape = (self.ndata['X'], self.nx)
            if self.dbin > 1:
                data['Y'] = np.array([data['Y'][i:i+self.dbin].mean() for i in range(0,self.ndata['X'],self.dbin)])
                data['X'] = np.array([[data['X'][i:i+self.dbin,j].mean() for j in range(self.nx)] 
                                           for i in range(0,self.ndata['X'],self.dbin)]).reshape(int(self.ndata['X']/self.dbin)*self.nx)
                self.ndata['Y'] = data['Y'].shape[0]
                self.ndata['X'] = self.ndata['Y']
            self.data = pds.DataFrame(data=data)
            del data
        return
    
    # Select data according to list of p, z, t, dt, return normalized data
    #   pzt: dictionary or list-like data chacacters of: 
    #        'p' (momentum), 'z' (z-link), 't' (Y time slices), 'dt' (X & Y time differences)
    #   tag: 'X' or 'Y'
    def select_data(self, pzt, tag = None):
        if self.form == 'raw':
            sdata = self.data.loc[:,tag].reshape(self.ndata['X'],self.nx)[:,pzt[1]::pzt[0]]
            smean = sdata.mean()
            sstd = sdata.std()
            sdata /= smean
            return None, None, sdata, smean, sstd
        # data form is 'pdf'
        if self.tdict[tag] is not None:
            del self.tdict[tag]
            self.tdict[tag] = None
        if self.pdict[tag] is not None:
            del self.pdict[tag]
        if self.zdict[tag] is not None:
            del self.zdict[tag]
        if isinstance(pzt, dict):
            pl = pzt['p']
            zl = pzt['z']
            tl = pzt['t']
            dtl = pzt['dt']
        else:
            pl = pzt[0]
            zl = pzt[1]
            tl = pzt[2]
            dtl = pzt[3]
        tl0 = tl
        if pl is None: 
            pl = self.p
        if zl is None:
            zl = self.z
        self.tdict[tag] = []
        self.pdict[tag] = list(pl)
        self.zdict[tag] = list(zl)
        # X data with dt range
        if dtl is not None:
            # maximum dt 
            if isinstance(dtl, int):
                rttag = (-dtl, dtl, 1)
                rdt = list(range(-dtl, dtl+1))
            # range or list of dt 
            elif isinstance(dtl, (tuple, list)):
                if isinstance(dtl, tuple):
                    try:
                        dt = dtl[2]
                    except:
                        dt = 1
                    rdt = list(range(dtl[0], dtl[1]+1))
                    rttag = (dtl[0], dtl[1], dt)
                else:
                    rdt = np.array(dtl).sort().tolist()
                    rttag = rdt
            else:
                print("Error: Unknown dt range dts_X !\n")
                sys.exit(1)
            self.tdict[tag] = list(rdt)
        nsrc = self.nsrc
        # choose entire NT range
        if tl is None:
            nsrc = 1
            tl = {}
            self.countXT = 0
            for k in self.NT:
                tl[k] = list(range(0,self.NT[k]))
        elif dtl is not None:
            ttl = []
            for t, dt in product(tl, dtl):
                ttl.append(t+dt)
            self.countXT = list(tl)
            tl = ttl
        ndata = np.array([self.ndata[str(p)+'+'+str(z)] for p,z in product(pl, zl)]).min()
        sdata = []
        ttag = []
        print(ndata)
        print(pl)
        print(zl)
        checkerr = False
        if (dtl is None) and (tl0 is None) and (self.emax is not None):
            checkerr = True
            rttag = {}
            self.tdict[tag] = {}
        flag = False
        if self.tdict[tag] is None:
            flag = True
        for p,z in product(pl, zl):
            tg = str(p)+'+'+str(z)
            if checkerr:
                self.tdict[tag][tg] = []
            if isinstance(tl, dict):
                tt = tl[tg]
            else: 
                tt = tl
            if checkerr:
                rttag[tg] = []
            for n, tmp in product(range(nsrc), tt):
                t = (tmp+n*int(self.NT[tg]/nsrc)+self.NT[tg]) % self.NT[tg]
                if tg+'+t+'+str(t)+'+R' in ttag:
                    continue
                datatmp = self.data.loc[:,[tg+'+t+'+str(t)+'+R',tg+'+t+'+str(t)+'+I']].T.values
                if checkerr:
                    arrtmp = np.array(datatmp)
                    if abs(arrtmp[0].std()/arrtmp[0].mean()) > self.emax and (abs(arrtmp[1].std()/arrtmp[1].mean()) > self.emax):
                        del arrtmp
                        del datatmp
                        rttag[tg].append(t)
                        continue
                    else:
                        self.tdict[tag][tg].append(t)
                elif flag:
                    self.tdict[tag].append(t)
                sdata.extend(datatmp)
                ttag.extend([tg+'+t+'+str(t)+'+R', tg+'+t+'+str(t)+'+I'])
                del datatmp
            flag = False
        arr = np.array(sdata)
        del sdata
        sh = list(arr.shape)
        tsh = 1
        for t in sh:
            tsh *= t
        smean = [ arr[i].mean() for i in range(len(arr)) ]
        sstd = [ arr[i].std() for i in range(len(arr)) ]
        for i in range(len(arr)):
            arr[i] /= smean[i]
       # return tuple(tgl), arr.reshape(sh).T, np.array(smean), np.array(sstd)
        return rttag, tuple(ttag), arr.T, np.array(smean), np.array(sstd)

    # Select X data with specified time range for a fit
    #   tagX: list of X id's
    #   dtX: range of time slices 
    #   tagY: the Y id
    #   X : list of X samples with len(tagX) variables
    def select_T(self, tagX, dtX, tagY, X = None):
        if X is None:
            aX = np.array(self.data.loc[:,list(tagX)].values)#.T
        else:
            aX = np.array(X).T
        assert(aX.shape[0] == len(tagX))
        nd = aX.shape[1]
        lX = []
        tgl = tagY.split('+')
        ty = tgl[3]
        if isinstance(dtX, tuple):
            tlist = list(range(dtX[0], dtX[1]+1, dtX[2]))
        else:
            tlist = dtX
        nnt = int(self.NT[tgl[0]+'+'+tgl[1]]/self.nsrc)
        for i in range(1, self.nsrc):
            tlist.extend(list(range(dtX[0]+i*nnt, dtX[1]+1+i*nnt, dtX[2])))
        for i in range(len(tlist)):
            tlist[i] = (tlist[i]+self.NT[tgl[0]+'+'+tgl[1]])%self.NT[tgl[0]+'+'+tgl[1]]
        for i in range(len(tagX)):
            if int(tagX[i].split('+')[3]) in tlist:
                lX.append(aX[i])
        return np.array(lX).T
    
    # Generate output directories and filenames
    def setup_output(self):
        self.date = datetime.datetime.today().strftime('%m%d%Y')
        if self.osave or self.dsave:
            odir = self.odir+'/ML.'+self.date
            sfix = ''
            n = 1
            flag = 0
            while True:
                try:
                    os.mkdir(odir+sfix)
                    flag = 1
                except:
                    sfix = '.'+str(n)
                    flag = 0
                    n += 1
                    #print("Warming: directory"+self.odir+'/ML.'+self.date+" already exist")
                if flag == 1:
                    break
            self.odir = odir+sfix
            if self.osave:
                try:
                    os.mkdir(self.odir+'/plots')
                except:
                    print("Warming: directory"+self.odir+'/plots'+" already exist")
            if self.dsave:
                try:
                    os.mkdir(self.odir+'/preds')
                except:
                    print("Warming: directory"+self.odir+'/preds'+" already exist")
            self.oheader_tag = None
        else:
            self.ofile = None
            self.pltfile = None
        return
    
    # Make the output file header lines
    #   indx: filename surfix (digital)
    def dfile_mkheader(self, indx):
        if self.dsave is False:
            return
        if self.form == 'raw':
            return
        if self.form == 'pdf':
            self.ofile = self.odir+'/'+self.ID+str(indx)
            self.pltfile = self.odir+'/plots/'+self.ID+str(indx)
            pf = open(self.ofile, 'a+')
            self.dfile_mkheader_pdf(pf)
            pf.close()
        return
    
    def dfile_mkheader_pdf(self, pf):
        if True: 
            pf.write("PDF data analysis with ML: \n Target data: ")
            n = 1
            for tgp, tgz in product(self.pdict['Y'], self.zdict['Y']):
                if self.NA == tgz: 
                    pf.write("Y{:d}( spectrum: p= {:} ),  ".format(n, tgp))
                else:
                    pf.write("Y{:d}( PDF/DA: p= {:} ,z= {:} ),  ".format(n, tgp, tgz))
                n+=1
            pl = []
            zl = []
            pf.write("\n Training data: p= {:} ,z= {:} \n".format(self.pdict['X'], self.zdict['X']))
            pf.write("\n Target data at t= {:} \n Training data at dt= {:} \n\n".format(self.tdict['Y'], 
                                                                                        self.tdict['X']))
        else:
            pf.write("\n Target data at t= {:} \n Training data at dt= {:} \n\n".format(self.tdict['Y'], 
                                                                                        self.tdict['X']))
        return
        